## Evaluation of Model Performance on Loss Functions

In [1]:
import os
import torch
import cv2
from bunch import Bunch
from ruamel.yaml import safe_load
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image

import torch.backends.cudnn as cudnn
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
# from sklearn.metrics import 
import sys
sys.path.append("../src/")
from tester import Tester
from utils import losses
from data.utils import clahe_equalized
import networks as models
from data.transforms import pipeline_tranforms

from utils.helpers import get_instance


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Dataset Class

In [3]:
class FIVES(Dataset):

    def __init__(self, data_path):
        super(FIVES, self).__init__()
        self.transforms = pipeline_tranforms()
        
        self.images_path = sorted(glob(data_path + "/test/Original/*"))
        self.masks_path  = sorted(glob(data_path + "/test/Ground truth/*"))

        self.n_samples = len(self.images_path)
    
    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = clahe_equalized(image)
        image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_NEAREST)

        image = image / 255.0  # type: ignore # (512, 512, 3) Normalizing to range (0,1)
        image = np.transpose(image, (2, 0, 1))  # (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        """ Reading mask """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,  (512, 512), interpolation=cv2.INTER_NEAREST)
        mask = mask / 255.0  # type: ignore # (512, 512)
        mask = np.expand_dims(mask, axis=0)  # (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)
        
        return image, mask

    def __len__(self):
        return self.n_samples
fives_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/FIVES'

fives_dataset = FIVES(fives_path)
fives_dataset.__len__()

200

In [4]:
def main_result(config_file, dataset, bs, weight_path):

    with open(config_file, encoding="utf-8") as file:
        CFG = Bunch(safe_load(file))
        
    checkpoint = torch.load(weight_path)
    CFG_ck = checkpoint['config']

    test_loader = DataLoader(dataset, bs ,shuffle=False,  
                            num_workers=CFG.num_workers, pin_memory=True)
    
    model = get_instance(models, 'model', CFG)
    loss = get_instance(losses, 'loss', CFG_ck)

    test = Tester(model, loss, CFG, checkpoint, test_loader, device)
    test.test()

## WNET 

In [5]:
# CE_DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/CE_DiceLoss/checkpoint-epoch100.pth'
config_file = '../configs/lwnet_dicebce.yaml'

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.2648 | AUC 0.9865 F1 0.8565 Acc 0.9806  Sen 0.8310 Spe 0.9915 Pre 0.8869 IOU 0.7508 MCC 0.8474 |B 0.21 D 0.07 |: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]
2023-08-07 09:22:20.196 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:22:20.197 | INFO     | tester:test:79 - test time:  0.2096
2023-08-07 09:22:20.198 | INFO     | tester:test:80 -      loss:  0.2648
2023-08-07 09:22:20.198 | INFO     | tester:test:84 - AUC  : 0.9865
2023-08-07 09:22:20.199 | INFO     | tester:test:84 - F1   : 0.8565
2023-08-07 09:22:20.199 | INFO     | tester:test:84 - Acc  : 0.9806
2023-08-07 09:22:20.200 | INFO     | tester:test:84 - Sen  : 0.831
2023-08-07 09:22:20.200 | INFO     | tester:test:84 - Spe  : 0.9915
2023-08-07 09:22:20.201 | INFO     | tester:test:84 - pre  : 0.8869
2023-08-07 09:22:20.201 | INFO     | tester:test:84 - IOU  : 0.7508
2023-08-07 09:22:20.202 | INFO     | tester:test:84 - MCC  : 0.8474


In [6]:
# DiceLoss

weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/DiceLoss/checkpoint-epoch100.pth'
config_file = '../configs/lwnet_dice.yaml'

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.3264 | AUC 0.9218 F1 0.8562 Acc 0.9805  Sen 0.8308 Spe 0.9915 Pre 0.8867 IOU 0.7505 MCC 0.8472 |B 0.07 D 0.05 |: 100%|██████████| 50/50 [00:35<00:00,  1.39it/s]
2023-08-07 09:23:11.041 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:23:11.042 | INFO     | tester:test:79 - test time:  0.0733
2023-08-07 09:23:11.043 | INFO     | tester:test:80 -      loss:  0.3264
2023-08-07 09:23:11.043 | INFO     | tester:test:84 - AUC  : 0.9218
2023-08-07 09:23:11.044 | INFO     | tester:test:84 - F1   : 0.8562
2023-08-07 09:23:11.045 | INFO     | tester:test:84 - Acc  : 0.9805
2023-08-07 09:23:11.045 | INFO     | tester:test:84 - Sen  : 0.8308
2023-08-07 09:23:11.046 | INFO     | tester:test:84 - Spe  : 0.9915
2023-08-07 09:23:11.046 | INFO     | tester:test:84 - pre  : 0.8867
2023-08-07 09:23:11.048 | INFO     | tester:test:84 - IOU  : 0.7505
2023-08-07 09:23:11.048 | INFO     | tester:test:84 - MCC  : 0.8472


In [7]:
# BCE_Loss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/BCELoss/checkpoint-epoch100.pth'
config_file = '../configs/lwnet_bce.yaml'

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1343 | AUC 0.9871 F1 0.8442 Acc 0.9791  Sen 0.8101 Spe 0.9915 Pre 0.8855 IOU 0.7323 MCC 0.8350 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:36<00:00,  1.38it/s]
2023-08-07 09:23:47.388 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:23:47.389 | INFO     | tester:test:79 - test time:  0.0779
2023-08-07 09:23:47.390 | INFO     | tester:test:80 -      loss:  0.1343
2023-08-07 09:23:47.391 | INFO     | tester:test:84 - AUC  : 0.9871
2023-08-07 09:23:47.391 | INFO     | tester:test:84 - F1   : 0.8442
2023-08-07 09:23:47.392 | INFO     | tester:test:84 - Acc  : 0.9791
2023-08-07 09:23:47.392 | INFO     | tester:test:84 - Sen  : 0.8101
2023-08-07 09:23:47.393 | INFO     | tester:test:84 - Spe  : 0.9915
2023-08-07 09:23:47.393 | INFO     | tester:test:84 - pre  : 0.8855
2023-08-07 09:23:47.394 | INFO     | tester:test:84 - IOU  : 0.7323
2023-08-07 09:23:47.394 | INFO     | tester:test:84 - MCC  : 0.835


In [8]:
# Soft-clDiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/Soft_clDice/checkpoint-epoch100.pth'
config_file = '../configs/lwnet_cldice.yaml'

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1625 | AUC 0.9262 F1 0.8567 Acc 0.9805  Sen 0.8344 Spe 0.9911 Pre 0.8835 IOU 0.7511 MCC 0.8475 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:36<00:00,  1.38it/s]
2023-08-07 09:24:23.719 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:24:23.720 | INFO     | tester:test:79 - test time:  0.0847
2023-08-07 09:24:23.720 | INFO     | tester:test:80 -      loss:  0.1625
2023-08-07 09:24:23.721 | INFO     | tester:test:84 - AUC  : 0.9262
2023-08-07 09:24:23.721 | INFO     | tester:test:84 - F1   : 0.8567
2023-08-07 09:24:23.722 | INFO     | tester:test:84 - Acc  : 0.9805
2023-08-07 09:24:23.722 | INFO     | tester:test:84 - Sen  : 0.8344
2023-08-07 09:24:23.724 | INFO     | tester:test:84 - Spe  : 0.9911
2023-08-07 09:24:23.724 | INFO     | tester:test:84 - pre  : 0.8835
2023-08-07 09:24:23.724 | INFO     | tester:test:84 - IOU  : 0.7511
2023-08-07 09:24:23.725 | INFO     | tester:test:84 - MCC  : 0.8475


## SA-UNet

In [9]:
# CE_DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/sa_unet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1693 | AUC 0.9854 F1 0.8655 Acc 0.9800  Sen 0.9134 Spe 0.9848 Pre 0.8234 IOU 0.7638 MCC 0.8564 |B 0.07 D 0.05 |: 100%|██████████| 50/50 [00:35<00:00,  1.39it/s]
2023-08-07 09:24:59.870 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:24:59.870 | INFO     | tester:test:79 - test time:  0.0729
2023-08-07 09:24:59.871 | INFO     | tester:test:80 -      loss:  0.1693
2023-08-07 09:24:59.871 | INFO     | tester:test:84 - AUC  : 0.9854
2023-08-07 09:24:59.872 | INFO     | tester:test:84 - F1   : 0.8655
2023-08-07 09:24:59.872 | INFO     | tester:test:84 - Acc  : 0.98
2023-08-07 09:24:59.873 | INFO     | tester:test:84 - Sen  : 0.9134
2023-08-07 09:24:59.873 | INFO     | tester:test:84 - Spe  : 0.9848
2023-08-07 09:24:59.874 | INFO     | tester:test:84 - pre  : 0.8234
2023-08-07 09:24:59.874 | INFO     | tester:test:84 - IOU  : 0.7638
2023-08-07 09:24:59.875 | INFO     | tester:test:84 - MCC  : 0.8564


In [10]:
# DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/sa_unet_dice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.2371 | AUC 0.9734 F1 0.8600 Acc 0.9791  Sen 0.9134 Spe 0.9838 Pre 0.8133 IOU 0.7552 MCC 0.8506 |B 0.07 D 0.05 |: 100%|██████████| 50/50 [00:35<00:00,  1.42it/s]
2023-08-07 09:25:35.246 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:25:35.247 | INFO     | tester:test:79 - test time:  0.0664
2023-08-07 09:25:35.248 | INFO     | tester:test:80 -      loss:  0.2371
2023-08-07 09:25:35.248 | INFO     | tester:test:84 - AUC  : 0.9734
2023-08-07 09:25:35.249 | INFO     | tester:test:84 - F1   : 0.86
2023-08-07 09:25:35.249 | INFO     | tester:test:84 - Acc  : 0.9791
2023-08-07 09:25:35.249 | INFO     | tester:test:84 - Sen  : 0.9134
2023-08-07 09:25:35.250 | INFO     | tester:test:84 - Spe  : 0.9838
2023-08-07 09:25:35.251 | INFO     | tester:test:84 - pre  : 0.8133
2023-08-07 09:25:35.251 | INFO     | tester:test:84 - IOU  : 0.7552
2023-08-07 09:25:35.252 | INFO     | tester:test:84 - MCC  : 0.8506


In [11]:
# BCELoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/BCELoss/checkpoint-epoch70.pth'
config_file = "../configs/sa_unet_bce.yaml"


main_result(config_file, dataset=fives_dataset,bs=4, weight_path=weight_path)

  0%|          | 0/50 [00:00<?, ?it/s]

TEST (49) | Loss: 0.0744 | AUC 0.9883 F1 0.8664 Acc 0.9806  Sen 0.8922 Spe 0.9870 Pre 0.8436 IOU 0.7653 MCC 0.8568 |B 0.07 D 0.05 |: 100%|██████████| 50/50 [00:35<00:00,  1.40it/s]
2023-08-07 09:26:11.150 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:26:11.151 | INFO     | tester:test:79 - test time:  0.0692
2023-08-07 09:26:11.153 | INFO     | tester:test:80 -      loss:  0.0744
2023-08-07 09:26:11.153 | INFO     | tester:test:84 - AUC  : 0.9883
2023-08-07 09:26:11.154 | INFO     | tester:test:84 - F1   : 0.8664
2023-08-07 09:26:11.154 | INFO     | tester:test:84 - Acc  : 0.9806
2023-08-07 09:26:11.155 | INFO     | tester:test:84 - Sen  : 0.8922
2023-08-07 09:26:11.155 | INFO     | tester:test:84 - Spe  : 0.987
2023-08-07 09:26:11.156 | INFO     | tester:test:84 - pre  : 0.8436
2023-08-07 09:26:11.156 | INFO     | tester:test:84 - IOU  : 0.7653
2023-08-07 09:26:11.157 | INFO     | tester:test:84 - MCC  : 0.8568


In [12]:
# Soft_clDice
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/Soft_clDice/checkpoint-epoch70.pth'
config_file = "../configs/sa_unet_cldice.yaml"


main_result(config_file, dataset=fives_dataset,bs=4, weight_path=weight_path)

  0%|          | 0/50 [00:00<?, ?it/s]

TEST (49) | Loss: 0.1183 | AUC 0.9739 F1 0.8623 Acc 0.9796  Sen 0.9098 Spe 0.9846 Pre 0.8206 IOU 0.7589 MCC 0.8530 |B 0.07 D 0.05 |: 100%|██████████| 50/50 [00:35<00:00,  1.40it/s]
2023-08-07 09:26:46.971 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:26:46.972 | INFO     | tester:test:79 - test time:  0.0736
2023-08-07 09:26:46.972 | INFO     | tester:test:80 -      loss:  0.1183
2023-08-07 09:26:46.973 | INFO     | tester:test:84 - AUC  : 0.9739
2023-08-07 09:26:46.973 | INFO     | tester:test:84 - F1   : 0.8623
2023-08-07 09:26:46.974 | INFO     | tester:test:84 - Acc  : 0.9796
2023-08-07 09:26:46.975 | INFO     | tester:test:84 - Sen  : 0.9098
2023-08-07 09:26:46.975 | INFO     | tester:test:84 - Spe  : 0.9846
2023-08-07 09:26:46.976 | INFO     | tester:test:84 - pre  : 0.8206
2023-08-07 09:26:46.976 | INFO     | tester:test:84 - IOU  : 0.7589
2023-08-07 09:26:46.977 | INFO     | tester:test:84 - MCC  : 0.853


## FR-UNet

In [13]:
# CE_DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/fr_unet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0829 | AUC 0.9911 F1 0.9037 Acc 0.9872  Sen 0.8686 Spe 0.9960 Pre 0.9430 IOU 0.8257 MCC 0.8980 |B 0.31 D 0.05 |: 100%|██████████| 50/50 [00:47<00:00,  1.06it/s]
2023-08-07 09:28:09.258 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:28:09.259 | INFO     | tester:test:79 - test time:  0.3146
2023-08-07 09:28:09.259 | INFO     | tester:test:80 -      loss:  0.0829
2023-08-07 09:28:09.260 | INFO     | tester:test:84 - AUC  : 0.9911
2023-08-07 09:28:09.260 | INFO     | tester:test:84 - F1   : 0.9037
2023-08-07 09:28:09.261 | INFO     | tester:test:84 - Acc  : 0.9872
2023-08-07 09:28:09.261 | INFO     | tester:test:84 - Sen  : 0.8686
2023-08-07 09:28:09.262 | INFO     | tester:test:84 - Spe  : 0.996
2023-08-07 09:28:09.262 | INFO     | tester:test:84 - pre  : 0.943
2023-08-07 09:28:09.262 | INFO     | tester:test:84 - IOU  : 0.8257
2023-08-07 09:28:09.263 | INFO     | tester:test:84 - MCC  : 0.898


In [14]:
# DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/fr_unet_dice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1091 | AUC 0.9597 F1 0.9004 Acc 0.9868  Sen 0.8639 Spe 0.9959 Pre 0.9418 IOU 0.8204 MCC 0.8947 |B 0.32 D 0.05 |: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s]
2023-08-07 09:29:09.156 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:29:09.157 | INFO     | tester:test:79 - test time:  0.315
2023-08-07 09:29:09.157 | INFO     | tester:test:80 -      loss:  0.1091
2023-08-07 09:29:09.158 | INFO     | tester:test:84 - AUC  : 0.9597
2023-08-07 09:29:09.158 | INFO     | tester:test:84 - F1   : 0.9004
2023-08-07 09:29:09.159 | INFO     | tester:test:84 - Acc  : 0.9868
2023-08-07 09:29:09.160 | INFO     | tester:test:84 - Sen  : 0.8639
2023-08-07 09:29:09.160 | INFO     | tester:test:84 - Spe  : 0.9959
2023-08-07 09:29:09.161 | INFO     | tester:test:84 - pre  : 0.9418
2023-08-07 09:29:09.161 | INFO     | tester:test:84 - IOU  : 0.8204
2023-08-07 09:29:09.162 | INFO     | tester:test:84 - MCC  : 0.8947


In [15]:
# BCELoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/BCELoss/checkpoint-epoch70.pth'
config_file = "../configs/fr_unet_bce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0413 | AUC 0.9928 F1 0.9014 Acc 0.9869  Sen 0.8675 Spe 0.9957 Pre 0.9398 IOU 0.8219 MCC 0.8957 |B 0.32 D 0.05 |: 100%|██████████| 50/50 [00:47<00:00,  1.05it/s]
2023-08-07 09:29:58.102 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:29:58.102 | INFO     | tester:test:79 - test time:  0.3164
2023-08-07 09:29:58.103 | INFO     | tester:test:80 -      loss:  0.0413
2023-08-07 09:29:58.103 | INFO     | tester:test:84 - AUC  : 0.9928
2023-08-07 09:29:58.104 | INFO     | tester:test:84 - F1   : 0.9014
2023-08-07 09:29:58.105 | INFO     | tester:test:84 - Acc  : 0.9869
2023-08-07 09:29:58.105 | INFO     | tester:test:84 - Sen  : 0.8675
2023-08-07 09:29:58.105 | INFO     | tester:test:84 - Spe  : 0.9957
2023-08-07 09:29:58.106 | INFO     | tester:test:84 - pre  : 0.9398
2023-08-07 09:29:58.106 | INFO     | tester:test:84 - IOU  : 0.8219
2023-08-07 09:29:58.106 | INFO     | tester:test:84 - MCC  : 0.8957


In [16]:
# Soft_clDice
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/Soft_clDice/checkpoint-epoch70.pth'
config_file = "../configs/fr_unet_cldice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0534 | AUC 0.9684 F1 0.9029 Acc 0.9870  Sen 0.8735 Spe 0.9954 Pre 0.9357 IOU 0.8242 MCC 0.8969 |B 0.31 D 0.05 |: 100%|██████████| 50/50 [00:47<00:00,  1.06it/s]
2023-08-07 09:30:46.255 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:30:46.255 | INFO     | tester:test:79 - test time:  0.3139
2023-08-07 09:30:46.256 | INFO     | tester:test:80 -      loss:  0.0534
2023-08-07 09:30:46.256 | INFO     | tester:test:84 - AUC  : 0.9684
2023-08-07 09:30:46.257 | INFO     | tester:test:84 - F1   : 0.9029
2023-08-07 09:30:46.258 | INFO     | tester:test:84 - Acc  : 0.987
2023-08-07 09:30:46.258 | INFO     | tester:test:84 - Sen  : 0.8735
2023-08-07 09:30:46.259 | INFO     | tester:test:84 - Spe  : 0.9954
2023-08-07 09:30:46.259 | INFO     | tester:test:84 - pre  : 0.9357
2023-08-07 09:30:46.260 | INFO     | tester:test:84 - IOU  : 0.8242
2023-08-07 09:30:46.260 | INFO     | tester:test:84 - MCC  : 0.8969


## MANet

In [17]:
# CE_DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/manet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0817 | AUC 0.9910 F1 0.8997 Acc 0.9864  Sen 0.8753 Spe 0.9947 Pre 0.9261 IOU 0.8186 MCC 0.8929 |B 0.10 D 0.05 |: 100%|██████████| 50/50 [00:34<00:00,  1.46it/s]
2023-08-07 09:32:47.124 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:32:47.125 | INFO     | tester:test:79 - test time:  0.0983
2023-08-07 09:32:47.125 | INFO     | tester:test:80 -      loss:  0.0817
2023-08-07 09:32:47.126 | INFO     | tester:test:84 - AUC  : 0.991
2023-08-07 09:32:47.126 | INFO     | tester:test:84 - F1   : 0.8997
2023-08-07 09:32:47.127 | INFO     | tester:test:84 - Acc  : 0.9864
2023-08-07 09:32:47.127 | INFO     | tester:test:84 - Sen  : 0.8753
2023-08-07 09:32:47.128 | INFO     | tester:test:84 - Spe  : 0.9947
2023-08-07 09:32:47.128 | INFO     | tester:test:84 - pre  : 0.9261
2023-08-07 09:32:47.129 | INFO     | tester:test:84 - IOU  : 0.8186
2023-08-07 09:32:47.129 | INFO     | tester:test:84 - MCC  : 0.8929


In [18]:
# DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/manet_dice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1047 | AUC 0.9605 F1 0.8988 Acc 0.9864  Sen 0.8680 Spe 0.9953 Pre 0.9327 IOU 0.8172 MCC 0.8923 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.48it/s]
2023-08-07 09:33:26.264 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:33:26.265 | INFO     | tester:test:79 - test time:  0.0829
2023-08-07 09:33:26.266 | INFO     | tester:test:80 -      loss:  0.1047
2023-08-07 09:33:26.266 | INFO     | tester:test:84 - AUC  : 0.9605
2023-08-07 09:33:26.267 | INFO     | tester:test:84 - F1   : 0.8988
2023-08-07 09:33:26.267 | INFO     | tester:test:84 - Acc  : 0.9864
2023-08-07 09:33:26.268 | INFO     | tester:test:84 - Sen  : 0.868
2023-08-07 09:33:26.268 | INFO     | tester:test:84 - Spe  : 0.9953
2023-08-07 09:33:26.269 | INFO     | tester:test:84 - pre  : 0.9327
2023-08-07 09:33:26.269 | INFO     | tester:test:84 - IOU  : 0.8172
2023-08-07 09:33:26.270 | INFO     | tester:test:84 - MCC  : 0.8923


In [19]:
# BCELoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/BCELoss/checkpoint-epoch70.pth'
config_file = "../configs/manet_bce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0402 | AUC 0.9937 F1 0.8941 Acc 0.9858  Sen 0.8604 Spe 0.9952 Pre 0.9318 IOU 0.8097 MCC 0.8876 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.49it/s]
2023-08-07 09:34:12.509 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:34:12.509 | INFO     | tester:test:79 - test time:  0.0848
2023-08-07 09:34:12.510 | INFO     | tester:test:80 -      loss:  0.0402
2023-08-07 09:34:12.510 | INFO     | tester:test:84 - AUC  : 0.9937
2023-08-07 09:34:12.512 | INFO     | tester:test:84 - F1   : 0.8941
2023-08-07 09:34:12.512 | INFO     | tester:test:84 - Acc  : 0.9858
2023-08-07 09:34:12.512 | INFO     | tester:test:84 - Sen  : 0.8604
2023-08-07 09:34:12.513 | INFO     | tester:test:84 - Spe  : 0.9952
2023-08-07 09:34:12.513 | INFO     | tester:test:84 - pre  : 0.9318
2023-08-07 09:34:12.514 | INFO     | tester:test:84 - IOU  : 0.8097
2023-08-07 09:34:12.514 | INFO     | tester:test:84 - MCC  : 0.8876


In [20]:
# Soft_clDice
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/Soft_clDice/checkpoint-epoch65.pth'
config_file = "../configs/manet_cldice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0521 | AUC 0.9684 F1 0.9005 Acc 0.9864  Sen 0.8813 Spe 0.9942 Pre 0.9212 IOU 0.8199 MCC 0.8936 |B 0.09 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.48it/s]
2023-08-07 09:34:51.156 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:34:51.157 | INFO     | tester:test:79 - test time:  0.0897
2023-08-07 09:34:51.157 | INFO     | tester:test:80 -      loss:  0.0521
2023-08-07 09:34:51.158 | INFO     | tester:test:84 - AUC  : 0.9684
2023-08-07 09:34:51.159 | INFO     | tester:test:84 - F1   : 0.9005
2023-08-07 09:34:51.159 | INFO     | tester:test:84 - Acc  : 0.9864
2023-08-07 09:34:51.161 | INFO     | tester:test:84 - Sen  : 0.8813
2023-08-07 09:34:51.161 | INFO     | tester:test:84 - Spe  : 0.9942
2023-08-07 09:34:51.162 | INFO     | tester:test:84 - pre  : 0.9212
2023-08-07 09:34:51.162 | INFO     | tester:test:84 - IOU  : 0.8199
2023-08-07 09:34:51.163 | INFO     | tester:test:84 - MCC  : 0.8936


## UNet

In [21]:
# CE_DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/unet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0800 | AUC 0.9914 F1 0.9015 Acc 0.9866  Sen 0.8838 Spe 0.9942 Pre 0.9208 IOU 0.8218 MCC 0.8947 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.48it/s]
2023-08-07 09:36:16.334 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:36:16.335 | INFO     | tester:test:79 - test time:  0.0834
2023-08-07 09:36:16.336 | INFO     | tester:test:80 -      loss:  0.08
2023-08-07 09:36:16.336 | INFO     | tester:test:84 - AUC  : 0.9914
2023-08-07 09:36:16.337 | INFO     | tester:test:84 - F1   : 0.9015
2023-08-07 09:36:16.337 | INFO     | tester:test:84 - Acc  : 0.9866
2023-08-07 09:36:16.338 | INFO     | tester:test:84 - Sen  : 0.8838
2023-08-07 09:36:16.338 | INFO     | tester:test:84 - Spe  : 0.9942
2023-08-07 09:36:16.339 | INFO     | tester:test:84 - pre  : 0.9208
2023-08-07 09:36:16.339 | INFO     | tester:test:84 - IOU  : 0.8218
2023-08-07 09:36:16.340 | INFO     | tester:test:84 - MCC  : 0.8947


In [22]:
# DiceLoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/unet_dice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.1052 | AUC 0.9652 F1 0.8987 Acc 0.9862  Sen 0.8800 Spe 0.9942 Pre 0.9193 IOU 0.8173 MCC 0.8919 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.50it/s]
2023-08-07 09:36:53.780 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:36:53.781 | INFO     | tester:test:79 - test time:  0.0831
2023-08-07 09:36:53.782 | INFO     | tester:test:80 -      loss:  0.1052
2023-08-07 09:36:53.782 | INFO     | tester:test:84 - AUC  : 0.9652
2023-08-07 09:36:53.782 | INFO     | tester:test:84 - F1   : 0.8987
2023-08-07 09:36:53.783 | INFO     | tester:test:84 - Acc  : 0.9862
2023-08-07 09:36:53.784 | INFO     | tester:test:84 - Sen  : 0.88
2023-08-07 09:36:53.785 | INFO     | tester:test:84 - Spe  : 0.9942
2023-08-07 09:36:53.785 | INFO     | tester:test:84 - pre  : 0.9193
2023-08-07 09:36:53.786 | INFO     | tester:test:84 - IOU  : 0.8173
2023-08-07 09:36:53.786 | INFO     | tester:test:84 - MCC  : 0.8919


In [23]:
# BCELoss
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/BCELoss/checkpoint-epoch70.pth'
config_file = "../configs/unet_bce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0399 | AUC 0.9937 F1 0.8976 Acc 0.9859  Sen 0.8835 Spe 0.9936 Pre 0.9129 IOU 0.8153 MCC 0.8904 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.50it/s]
2023-08-07 09:37:31.220 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:37:31.221 | INFO     | tester:test:79 - test time:  0.0813
2023-08-07 09:37:31.221 | INFO     | tester:test:80 -      loss:  0.0399
2023-08-07 09:37:31.222 | INFO     | tester:test:84 - AUC  : 0.9937
2023-08-07 09:37:31.222 | INFO     | tester:test:84 - F1   : 0.8976
2023-08-07 09:37:31.223 | INFO     | tester:test:84 - Acc  : 0.9859
2023-08-07 09:37:31.223 | INFO     | tester:test:84 - Sen  : 0.8835
2023-08-07 09:37:31.224 | INFO     | tester:test:84 - Spe  : 0.9936
2023-08-07 09:37:31.224 | INFO     | tester:test:84 - pre  : 0.9129
2023-08-07 09:37:31.225 | INFO     | tester:test:84 - IOU  : 0.8153
2023-08-07 09:37:31.225 | INFO     | tester:test:84 - MCC  : 0.8904


In [24]:
# Soft_clDice
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/Soft_clDice/checkpoint-epoch70.pth'
config_file = "../configs/unet_cldice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0515 | AUC 0.9661 F1 0.9006 Acc 0.9864  Sen 0.8850 Spe 0.9940 Pre 0.9178 IOU 0.8202 MCC 0.8938 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:33<00:00,  1.50it/s]
2023-08-07 09:38:09.813 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-07 09:38:09.814 | INFO     | tester:test:79 - test time:  0.082
2023-08-07 09:38:09.814 | INFO     | tester:test:80 -      loss:  0.0515
2023-08-07 09:38:09.814 | INFO     | tester:test:84 - AUC  : 0.9661
2023-08-07 09:38:09.815 | INFO     | tester:test:84 - F1   : 0.9006
2023-08-07 09:38:09.815 | INFO     | tester:test:84 - Acc  : 0.9864
2023-08-07 09:38:09.816 | INFO     | tester:test:84 - Sen  : 0.885
2023-08-07 09:38:09.817 | INFO     | tester:test:84 - Spe  : 0.994
2023-08-07 09:38:09.817 | INFO     | tester:test:84 - pre  : 0.9178
2023-08-07 09:38:09.818 | INFO     | tester:test:84 - IOU  : 0.8202
2023-08-07 09:38:09.818 | INFO     | tester:test:84 - MCC  : 0.8938
